In [1]:
import ast
import copy
import numpy as np
import os

In [2]:
import cc.atlas as atlas
import cc.cartography as cartography
import cc.publication as publication
import cc.utils as utils
import cc.tex as tex

In [3]:
import trove

# Literature Review

This notebook contains a record of how I performed the literature review for this work.

# Parameters

In [4]:
config_fp = '/Users/zhafen/paper_repos/cgm_modeling_challenge_paper/literature_review/literature_review.trove'
cp = trove.config_parser.ConfigParser( config_fp )
pm = trove.link_params_to_config(
    config_fp,
    variation = 'multicloud_modeling',
)

In [5]:
atlas_dir = pm['root_data_dir']

In [6]:
topics = {}
for variation in cp.variations:
    topics[variation] = ast.literal_eval( cp.get( variation, 'publications' ) )

# Setup

# Extensive Survey
Get all papers that might be of relevance.
We'll later identify the subset that are of more interest.

## Base Library
My existing library of papers.

In [7]:
a_zotero = atlas.Atlas( atlas_dir, data_fp=pm['zotero_atlas_fp'], load_bibtex=False, )

Loading saved atlas data.


0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2395/2395 [00:00<00:00, 17028.20it/s]


In [ ]:
a_zotero.import_bibtex( pm['zotero_bibtex_fp'] )

In [ ]:
a_zotero.process_abstracts( identifier='from_citation' )

In [ ]:
a_zotero.save_data( fp=pm['zotero_atlas_fp'] )

In [8]:
# Vectorize
vp_dict = a_zotero.vectorize( projection_fp=pm['zotero_projection_fp'], overwrite=True )
c_zotero = cartography.Cartographer( **vp_dict )

Vectorizing text...
    Retrieving publication data...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2395/2395 [00:00<00:00, 308641.60it/s]


    Calculating vectorization...


/Users/zhafen/miniconda3/envs/general_use/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 43.01it/s]


## Search Centered on Abstract
Perform a search centered on the abstract of the paper we are writing.

In [9]:
pub_doc = tex.Tex( filepath=pm['tex_fp'] )

In [10]:
abstract = pub_doc.string.split( '\\begin{abstract}' )[1].split( '\\end{abstract}' )[0]

In [11]:
print( abstract )


In the Halo21 circumgalactic medium (CGM) modeling challenge we presented observational absorption-system experts with three blinded sets of synthetic observations and asked them to derive the properties of the associated gas.
Our results reveal that common assumptions about ionization equilibrium and cloud structure can lead to $\gtrsim 1$ dex errors in estimated metallicity, temperature, or density.
The three data samples were generated and analyzed iteratively with increasing complexity.
\texttt{sample0} was ion column densities of uniform clouds, and revealed that constraining models to either collisional or photo-ionization equilibrium results in best estimates of metallicity being off by up to $\sim 1$ dex.
\texttt{sample1} was mock absorption spectra of multi-cloud systems, and demonstrated that the observational modeling of individual clouds could straightforwardly be expanded to absorption systems produced by one to three clouds.
\texttt{sample2} was spectra of a high-resolut

### Add Abstract to Atlas

In [12]:
p = publication.UnofficialPublication( pm['citation_key'] )

In [14]:
p.process_abstract( abstract_str = abstract )

In [15]:
a_zotero.data[p.citation_key] = copy.deepcopy( p )

In [16]:
# Vectorize
vp_dict = a_zotero.vectorize( projection_fp=pm['zotero_projection_fp'], overwrite=True )
c_zotero = cartography.Cartographer( **vp_dict )

Vectorizing text...
    Retrieving publication data...


  0%|                                                                                                                                                                                            | 0/2396 [00:00<?, ?it/s]/Users/zhafen/repos/cc/cc/publication.py:800: UserWarning: Hafen2022b is unofficial and has no publication date.
  warnings.warn( warning_msg )
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2396/2396 [00:00<00:00, 65723.30it/s]

    Calculating vectorization...



/Users/zhafen/miniconda3/envs/general_use/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 60.63it/s]


### Perform the Search

#### First Search

In [17]:
a = c_zotero.expand( a_zotero, center=p.citation_key )

Expansion will include 3620 new publications.
Loading bibliography entries.
Storing bibliography entries.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3620/3620 [00:00<00:00, 7460.21it/s]


In [18]:
a.process_abstracts()
a.save_data()

    Making 42 ADS calls...


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋    | 41/42 [01:56<00:03,  3.10s/it]/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Anderson2016. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Azoulay2011. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for CANaturalResourcesAgency. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Cen2001. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Chan2017. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Chawla2021. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No public

    Making 0 ADS calls for publications without IDs...


0it [00:00, ?it/s]


    Doing NLP...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6016/6016 [02:37<00:00, 38.10it/s]


Preparing to save data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6016/6016 [00:00<00:00, 210283.20it/s]


Saving to /Users/zhafen/Data/cgm_modeling_challenge/literature_review/atlas_data.json


#### Second Search

In [19]:
# Vectorize
vp_dict = a.vectorize( overwrite=True )
c = cartography.Cartographer( **vp_dict )

Vectorizing text...
    Retrieving publication data...


  0%|                                                                                                                                                                                            | 0/6016 [00:00<?, ?it/s]/Users/zhafen/repos/cc/cc/publication.py:800: UserWarning: Hafen2022b is unofficial and has no publication date.
  warnings.warn( warning_msg )
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6016/6016 [00:00<00:00, 133114.58it/s]

    Calculating vectorization...



/Users/zhafen/miniconda3/envs/general_use/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 25.99it/s]


In [20]:
a = c.expand( a, center=p.citation_key )

Expansion will include 3295 new publications.
Loading bibliography entries.
Storing bibliography entries.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6915/6915 [00:00<00:00, 37422.42it/s]


In [21]:
a.process_abstracts()
a.save_data()

    Making 39 ADS calls...


 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋         | 37/39 [01:30<00:04,  2.29s/it]/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Anderson2016. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Azoulay2011. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for CANaturalResourcesAgency. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Cen2001. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Chan2017. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Chawla2021. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No public

    Making 0 ADS calls for publications without IDs...


0it [00:00, ?it/s]


    Doing NLP...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9311/9311 [02:02<00:00, 75.86it/s]


Preparing to save data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9311/9311 [00:00<00:00, 174039.91it/s]


Saving to /Users/zhafen/Data/cgm_modeling_challenge/literature_review/atlas_data.json


#### Third Search

In [22]:
# Vectorize
vp_dict = a.vectorize( overwrite=True )
c = cartography.Cartographer( **vp_dict )

Vectorizing text...
    Retrieving publication data...


  0%|                                                                                                                                                                                            | 0/9311 [00:00<?, ?it/s]/Users/zhafen/repos/cc/cc/publication.py:800: UserWarning: Hafen2022b is unofficial and has no publication date.
  warnings.warn( warning_msg )
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9311/9311 [00:00<00:00, 175486.72it/s]

    Calculating vectorization...



/Users/zhafen/miniconda3/envs/general_use/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 17.51it/s]


In [23]:
a = c.expand( a, center=p.citation_key )

Expansion will include 3589 new publications.
Loading bibliography entries.
Storing bibliography entries.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10504/10504 [00:00<00:00, 39789.54it/s]


In [24]:
a.process_abstracts()
a.save_data()

    Making 42 ADS calls...


 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍        | 40/42 [01:33<00:05,  2.63s/it]/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Anderson2016. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Azoulay2011. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for CANaturalResourcesAgency. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Cen2001. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Chan2017. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No publications found for Chawla2021. Skipping.
  warnings.warn(
/Users/zhafen/repos/cc/cc/atlas.py:998: UserWarning: No public

    Making 0 ADS calls for publications without IDs...


0it [00:00, ?it/s]


    Doing NLP...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12900/12900 [02:12<00:00, 97.16it/s]


Preparing to save data.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12900/12900 [00:00<00:00, 165965.83it/s]


Saving to /Users/zhafen/Data/cgm_modeling_challenge/literature_review/atlas_data.json


#### Finish Up

In [25]:
# Vectorize
vp_dict = a.vectorize( overwrite=True )
c = cartography.Cartographer( **vp_dict )

Vectorizing text...
    Retrieving publication data...


  0%|                                                                                                                                                                                           | 0/12900 [00:00<?, ?it/s]/Users/zhafen/repos/cc/cc/publication.py:800: UserWarning: Hafen2022b is unofficial and has no publication date.
  warnings.warn( warning_msg )
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12900/12900 [00:00<00:00, 208597.02it/s]

    Calculating vectorization...



/Users/zhafen/miniconda3/envs/general_use/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.33it/s]


In [26]:
print( 'We now have {} related publications to search through!'.format( c.publications.size ) )

We now have 11896 related publications to search through!
